# Human Detection Project

In [141]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
from sklearn.decomposition import PCA

# Lectura de imagenes

In [5]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [6]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [7]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroCortesxPh = 20      # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

In [8]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

# Extracción de Caracterísitcas
    L1
       Normalization using L1-norm.
    L1-sqrt
       Normalization using L1-norm, followed by square root.
    L2
       Normalization using L2-norm.
    L2-Hys
       Normalization using L2-norm, followed by limiting the
       maximum values to 0.2 (`Hys` stands for `hysteresis`) and
       renormalization using L2-norm. (default)

In [132]:
orientations    = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)
block_norm      = 'L2'   
visualize       = False
visualise       = None
transform_sqrt  = True
feature_vector  = True
multichannel    = None

In [133]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector,multichannel))

    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i,:] = hog(imagenes[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector,multichannel)
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [134]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

In [135]:
fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

In [152]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

print(fotosTrain.shape)

(4563, 6804)


### PCA 

In [164]:
n_components= fotosTrain.shape[1]//2
pca = PCA(n_components)
dataPca = pca.fit_transform(fotosTrain)

In [165]:
dataPca.shape

(4563, 3402)

# Entrenamiento del modelo

In [166]:
clf = svm.LinearSVC(C = 0.007)
clf.fit(dataPca,y.ravel())

LinearSVC(C=0.007, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [168]:
#clf.score(fdPosTrain,yPos)
datPosTrain = pca.transform(fdPosTrain)
clf.score(datPosTrain,yPos)

0.9715808170515098

# Slide Window

In [178]:
def secImagen(imagen,maskHeight = 134,maskWidth = 70,step = 10):
    arrayImagen = {}
    aux = 0
    for i in range(0,imagen.shape[0]-maskHeight,step):        
        for j in range(0,imagen.shape[1]-maskWidth,step):
            arrayImagen[aux] = imagen[i:i+maskHeight, j:j+maskWidth]
            aux = aux + 1
    return arrayImagen

In [179]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotostest = leerFotos(path)

In [180]:
partImagen = secImagen(fotostest[6])

# Prueba del Modelo

In [170]:
listOk = []
for i in range(len(partImagen)):
    pruebaHog = hog(partImagen[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector,multichannel);
    prueba = pca.transform(pruebaHog.reshape(1,-1))
    if (1 == clf.predict(prueba.reshape(1,-1))):
        listOk.append(i);

In [172]:
cv2.imshow('img ', fotostest[6])
cv2.imshow('xd ', partImagen[113])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [171]:
print(len(listOk))
print(listOk)

55
[113, 291, 503, 836, 926, 1104, 1139, 1160, 1187, 1220, 1252, 1281, 1557, 1571, 1581, 1585, 1597, 1620, 1627, 1629, 1631, 1632, 1651, 1667, 1670, 1690, 1701, 1721, 1726, 1769, 1771, 1808, 1841, 1856, 1857, 1908, 1938, 1939, 1944, 1978, 2014, 2081, 2120, 2127, 2151, 2190, 2197, 2221, 2267, 2285, 2291, 2407, 2557, 2617, 2697]
